In [1]:
# 예제 9-2
from array import array
import math


class Vector2d:
    typecode = 'd'  # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):  # Vector2d를 반복할 수 있게 만들어서, 튜플 언패킹 및 제너레이터 표현식을 할 수 있게 만든다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

In [2]:
# 예제 9-1 다양하게 표현되는 Vector2d 객체
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

x, y = v1
print(x, y)
print(v1)
v1

3.0 4.0
3.0 4.0
(3.0, 4.0)


Vector2d(3.0, 4.0)

In [3]:
v1_clone = eval(repr(v1))
print("v1 == v1_clone:", v1 == v1_clone)

octets = bytes(v1)
print(octets)

print(abs(v1))
print(bool(v1), bool(Vector2d(0, 1)))

v1 == v1_clone: True
b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'
5.0
True True


## 9.3 대안 생성자
- `bytes`를 `Vector2d`로 변환하는 방법

In [4]:
# 예제 9-2
from array import array
import math


class Vector2d:
    typecode = 'd'  # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):  # Vector2d를 반복할 수 있게 만들어서, 튜플 언패킹 및 제너레이터 표현식을 할 수 있게 만든다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

## 9.4 @classmethod와 @staticmethod
- `@classmethod`: 객체가 아닌 클래스에 연산을 수행하는 메서드
  - 클래스 자체를 첫 번째 인수로 받음
  - 대안 생성자 (객체를 생성)를 구현할 때 주로 사용
- `@staticmethod`: 특별한 첫 번째 인수를 받지 않도록 변경된 메서드
  - 모듈 대신클래스 본체 안에 정의된 평범한 함수

In [5]:
# 예제 9-4 @classmethod와 @staticmethod
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    @staticmethod
    def statmeth(*args):
        return args
    
print(Demo.klassmeth())
print(Demo.klassmeth('spam'))
print(Demo.statmeth())
print(Demo.statmeth('spam'))

(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'spam')
()
('spam',)


## 9.5 포맷된 출력
- `format` 내장 함수와 `str.format()` 메서드는 실제 포맷 작업을 `__format__(format_spec)` 메서드에 위임한다.
- `format_spec`은 포맷 명시자로서 다음 두 가지 방법 중 하나를 통해 지정
  - `format(my_obj, format_spec)`의 두 번째 인수
  - `str.format()`에 사용된 포맷 문자열 안에 `{}`로 구분한 대체 필드 안에서 콜론 뒤의 문자열
    - 포맷 문자열 `{rate:0.2f}`에서 `:` 왼쪽은 대체 필드 구문에서 필드명에 해당하고, 오른쪽은 포맷 명시자이다.

In [6]:
brl = 1/2.43
print(brl)

print(format(brl, '0.4f'))  # '0.4f'가 포맷 명시자

print('1 BRL = {rate:0.2f} USD'.format(rate=brl))  # '0.2f'가 포맷 명시자

0.4115226337448559
0.4115
1 BRL = 0.41 USD


포맷 명시 간이 언어 예시
- `int`형
  - `b`: 이진수, `x`: 16진수
- `float`형
  - `f`:고정소수점, `%` 백분율

In [7]:
print(format(42, 'b'))  # 42의 이진수
print(format(2/3, '.1%'))  # 2/3의 백분율

101010
66.7%


- 클래스에서 `__format__()` 메서드를 구현하여 원하는대로 포맷 명시 간이 언어를 확장할 수 있다.

In [8]:
from datetime import datetime

now = datetime.now()
~print(format(now, '%H:%M:%S'))  # datetime 객체의 __format__()을 부르겠지?

00:04:41


TypeError: bad operand type for unary ~: 'NoneType'

`__format__()` 메서드가 정의되어 있지 않은 경우 `str(객체)`를 반환

In [9]:
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

In [10]:
format(v1, '.3f')

TypeError: unsupported format string passed to Vector2d.__format__

In [11]:
# 예제 9-5 Vector2d.format() 버전1
from array import array
import math


class Vector2d:
    typecode = 'd'  # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):  # Vector2d를 반복할 수 있게 만들어서, 튜플 언패킹 및 제너레이터 표현식을 할 수 있게 만든다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        components = (format(c, fmt_spec) for c in self)
        return '({}, {})'.format(*components)
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [12]:
v1 = Vector2d(3, 4)
print(format(v1))
print(format(v1, '.2f'))
print(format(v1, '.3e'))

(3.0, 4.0)
(3.00, 4.00)
(3.000e+00, 4.000e+00)


사용자 정의 포맷 명시 간이 언어 만들기
- `p`: 극좌표 $(r, \theta)$로 표현

In [13]:
# 예제 9-5 Vector2d.format() 버전2
from array import array
import math


class Vector2d:
    typecode = 'd'  # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):  # Vector2d를 반복할 수 있게 만들어서, 튜플 언패킹 및 제너레이터 표현식을 할 수 있게 만든다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]  # p 뗴어내기
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)

In [14]:
x = Vector2d(1, 1)
print(format(x, 'p'))
print(format(x, '.3ep'))
print(format(x, '0.5fp'))

<1.4142135623730951, 0.7853981633974483>
<1.414e+00, 7.854e-01>
<1.41421, 0.78540>


## 9.6 해시 가능한 Vector2d
- 해시 가능하게만들어주면 집합의 항목으로 사용하거나, 딕셔너리의 키로 사용할 수 있게 된다.

In [15]:
v1 = Vector2d(3, 4)

hash(v1)

TypeError: unhashable type: 'Vector2d'

In [16]:
{v1: 3}

TypeError: unhashable type: 'Vector2d'

- 해시 가능하게 만들려면, (111pg)
  - `__hash__()` 메서드와 `__eq__()` 메서드를 구현해야 한다. (<- 사실 이것만 정의해도 `hash()` 내장 함수가 잘 작동하는데, 객체의 해쉬값이 변하는 것을 방지하기 위해 불변형으로 만들어준다.)
  - 불변형 객체로 만들어야 한다.

In [17]:
# 예제 9-7 여기서는 Vector2d를 불변형으로 만든다.
from array import array
import math


class Vector2d:
    typecode = 'd'  # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property              #  `@property` 데커레이터는 프로퍼티의 게터 메서드를 나타낸다.
    def x(self):           # 공개할 속성명으로 메서드 이름을 설정한다.
        return self.__x    # self.__x를 반환한다.
    
    @property
    def y(self):
        return self.__y
        
    def __iter__(self):  # Vector2d를 반복할 수 있게 만들어서, 튜플 언패킹 및 제너레이터 표현식을 할 수 있게 만든다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]  # p 뗴어내기
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)

In [18]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

v1.x = 7

3.0 4.0


AttributeError: can't set attribute

`__hash__()` 메서드 구현
- int형을 반환해야 한다.
- 동일하다고 판단되는 객체는 동일한 해시값을 가져와야 한다.
  - `__eq__()` 메서드가 사용하는 객체의 속성을 이용해서 해시를 계산하는 것이 이상적이다.
- 시퀀스형의 각 원소에 비트 단위 XOR 연산자를 사용하는 것을 권장한다.

In [19]:
# 예제 9-8 __hash__() 메서드 구현
from array import array
import math


class Vector2d:
    typecode = 'd'  # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property              #  `@property` 데커레이터는 프로퍼티의 게터 메서드를 나타낸다.
    def x(self):           # 공개할 속성명으로 메서드 이름을 설정한다.
        return self.__x    # self.__x를 반환한다.
    
    @property
    def y(self):
        return self.__y
        
    def __iter__(self):  # Vector2d를 반복할 수 있게 만들어서, 튜플 언패킹 및 제너레이터 표현식을 할 수 있게 만든다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]  # p 뗴어내기
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)

In [20]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)

print(hash(v1), hash(v2))

print(set([v1, v2]))

7 384307168202284039
{Vector2d(3.1, 4.2), Vector2d(3.0, 4.0)}


내장 함수 `int`와 `float`과 호환되기 위하여 `__int__()`와 `__float__()` 구현

In [21]:
v1 = Vector2d(3, 4)
int(v1)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'Vector2d'

In [22]:
# 예제 9-8 __hash__() 메서드 구현
from array import array
import math


class Vector2d:
    typecode = 'd'  # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property              #  `@property` 데커레이터는 프로퍼티의 게터 메서드를 나타낸다.
    def x(self):           # 공개할 속성명으로 메서드 이름을 설정한다.
        return self.__x    # self.__x를 반환한다.
    
    @property
    def y(self):
        return self.__y
        
    def __iter__(self):  # Vector2d를 반복할 수 있게 만들어서, 튜플 언패킹 및 제너레이터 표현식을 할 수 있게 만든다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]  # p 뗴어내기
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    def __int__(self):
        x, y = self.x, self.y
        return self.__class__(int(x), int(y))
     
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)

## 9.7 파이썬에서의 비공개 속성과 보호된 속성
- 비공개 성격의 속성을 실수로 변경하지 못하게 하는 메커니즘이 있다.
  - 2개의 언더바로 시작하고, 언더바 없이 또는 하나의 언더바로 끝나도록 정의하면, 이름 장식을 붙여서 `_classname.__variablename` 형태로 `__dict__`에 저장한다.
  - 사용 예) 부모 클래스에 있는 속성이 자식 클래스에서 바뀔 위험성이 있을 때
  - 실수로 접근하는 것만 막아주고, 고의적인 악용은 막지 못한다.

In [23]:
v1 = Vector2d(3, 4)
print(v1.__dict__)
print(v1._Vector2d__x)

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}
3.0


바꾼 줄 알았지만 근본은 바뀌지 않았다.

In [24]:
v1.__x = 7
print(v1)
print(v1.x)
print(v1.__x)
print(v1.__dict__)

(3.0, 4.0)
3.0
7
{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0, '__x': 7}


In [25]:
v1 = Vector2d(3, 4)
v1._Vector2d__x = 7
print(v1.__x)

AttributeError: 'Vector2d' object has no attribute '__x'

In [26]:
print(v1)
print(v1.x)
print(v1.__dict__)

(7, 4.0)
7
{'_Vector2d__x': 7, '_Vector2d__y': 4.0}


`__x` 이거 쓰지 말고, 처음부터 직접 이름 장식 사용해서 정의하는 것이 낫다.

In [27]:
class MyClass1:
    def __init__(self):
        self.__x = 1
        
    @property
    def x(self):
        return self.__x
    
cls1 = MyClass1()
cls1.x

1

In [28]:
class MyClass2:
    def __init__(self):
        self._MyClass2__x = 1
        
    @property
    def x(self):
        return self.__x
    
cls2 = MyClass2()
cls2.x

1

- 보호된 속성: 언더바 하나만 작성하는 것은 파이썬 인터프리터가 별도 처리하지 않지만, 해당 속성에 접근하지 않는 것이 파이썬 프로그래머 사이에 일종의 국룰이다.
- 파이썬에는 진정한 비공개 속성과 불변 속성이 없다.

## 9.8 `__slots__` 클래스 속성으로 공간 절약하기
- 객체의 속성을 `__dict__` 속성에 저장하는데, `__dict__`는 딕셔너리 자료형이다. 딕셔너리 자료형은 해시 테이블을 유지해야 하기 때문에 메모리 사용량이 크다.
- 객체의 속성이 많지 않을 경우 클래스 속성 `__slots__`을 사용하여 속성을 튜플에 저장할 수 있다.
- (참고) 각 클래스에서 개별적으로 정의되는 `__slots__`을 고려한다. 상속받은 `__slots__`을 고려하지 않는다.
- (사용법) `__slots__`를 클래스 속성으로 만들고, 튜플에 저장하고 싶은 속성의 이름을 담은 시퀀스형을 할당하면 된다.
  - 시퀀스형 중 `__slots__`를 변경하지 못하도록 튜플을 사용한다.
- (주의) `__slots__`을 사용하는 순간부터 해당 객체는  `__slots__`에 명시된 속성 이외의 속성은 갖지 못한다.

In [29]:
v1 = Vector2d(3, 4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [30]:
# 예제 9-8 __hash__() 메서드 구현
from array import array
import math


class Vector2d:
    __slots__ = ('__x', '__y')
    typecode = 'd'  # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property              #  `@property` 데커레이터는 프로퍼티의 게터 메서드를 나타낸다.
    def x(self):           # 공개할 속성명으로 메서드 이름을 설정한다.
        return self.__x    # self.__x를 반환한다.
    
    @property
    def y(self):
        return self.__y
        
    def __iter__(self):  # Vector2d를 반복할 수 있게 만들어서, 튜플 언패킹 및 제너레이터 표현식을 할 수 있게 만든다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]  # p 뗴어내기
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    def __int__(self):
        x, y = self.x, self.y
        return self.__class__(int(x), int(y))
     
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)

In [31]:
v1 = Vector2d(3, 4)
v1.__dict__

AttributeError: 'Vector2d' object has no attribute '__dict__'

## 9.9 클래스 속성 오버라이드
- 클래스 속성과 동일한 이름을 갖는 속성에 할당을 할 경우 그 후부터 객체의 속성을 불러서 사용하게 된다.

In [32]:
# 예제 9-8 __hash__() 메서드 구현
from array import array
import math


class Vector2d:
    typecode = 'd'  # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property              #  `@property` 데커레이터는 프로퍼티의 게터 메서드를 나타낸다.
    def x(self):           # 공개할 속성명으로 메서드 이름을 설정한다.
        return self.__x    # self.__x를 반환한다.
    
    @property
    def y(self):
        return self.__y
        
    def __iter__(self):  # Vector2d를 반복할 수 있게 만들어서, 튜플 언패킹 및 제너레이터 표현식을 할 수 있게 만든다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]  # p 뗴어내기
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    def __int__(self):
        x, y = self.x, self.y
        return self.__class__(int(x), int(y))
     
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)

In [33]:
# 예제 9-13 클래스에서 상속받은 typecode 속성을 설정해서 객체 커스터마이즈 하기
v1 = Vector2d(1.1, 2.2)
dumpd = bytes(v1)
print(dumpd)
print(len(dumpd))

v1.typecode = 'f'
dumpf = bytes(v1)
print(dumpf)
print(len(dumpf))

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'
17
b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'
9


- 클래스 속성을 변경하려면 클래스 정의에서 직접 바꿔야 하며, 객체를통해 변경하면 안된다.
- 클래스 정의에서 바꾸지 않고, 다음과 같이 클래스 속성을 변경하면, 모든 객체의 기본 typecode도 바뀐다.
  - self.typecode나 객체.typecode를 실행할 경우 Vector2d.typecode를 콜하기 때문이다.

In [34]:
v2 = Vector2d(3, 4)
print(v2.typecode)

Vector2d.typecode = 'f'
print(v2.typecode)

d
f


- 클래스 속성을 다음과 같이 다룰 수 있다.

In [35]:
# 예제 9-14 typecode만 덮어쓴 서브 클래스
class ShortVector2d(Vector2d):
    typecode = 'f'

v1 = Vector2d(3, 4)
print(v1.typecode)

f
